# Fonctions pour les paramètres dérivés.

## Paramètres communs aux différentes fonctions 
La fonction prends en entrée 2 paramètres : 
1. ***psd*** : pandas.core.frame.DataFrame, *PSD en **#/L** à partir de laquelle calculer le paramètre dérivé.*
2. ***bin_size*** : list ou pandas.core.series.Series, _la taille du bin en **micro-mètre**, si la taille est constante alors **bin_size = [C]**. Si la taille des bins n'est pas constante alors la list/Serie doit être de la même taille que le nombre de colonne du DataFrame. Une erreur est retournée si un autre type est donné pour bin_size._ 

2 paramètres optionnels : 
1. ***bin_start, bin_end*** : int, default = None, _choix du bin de départ si on veut commencer à calculer le paramètre à partir d'une taille précise. Par exemple : si on veut calculer la concentration à partir de 100 µm pour la 2DS alors bin_start=10._ 

Cette structure est réutilisée pour **Concentration, LWC, IWC, Extinction et Effective_Diameter**.

## Sortie des fonctions 
* Concentration : panda.core.serie.Series, concentration totale avec le temps en index. 
* Diamètre effectif : panda.core.serie.Series, diamètre effectif en **µm** avec le temps en index. 
* LWC : panda.core.frame.DataFrame, Contenu en eau liquide $(g.m^{-3})$ par bin avec le temps en index.
* IWC : panda.core.frame.DataFrame, distribution en masse solide $(g.m^{-3})$ par bin avec le temps en index.
* Extinction : panda.core.frame.DataFrame, extinction en **$km^{-1}$** par bin avec le temps en index.

Pour LWC, IWC et Extinction, il suffit de sommer pour revenir à un LWC total ou un IWC _(LWC.sum(axis=1))_.

In [5]:
def plot(value):
    fig, ax = plt.subplots()
    ax.plot(value.index, value)
    plt.show()

def Concentration(psd, bin_size, bin_start = None, bin_end = None):
    '''
    psd : pd.DataFrame, index : temps, colonne : bin.
    bin_size : pd.Series() ou list, permet de passer de #/Volume/taille à #/Volume. Correspond à la différence (upper_bin_edge-lower_bin_edge).
    bin_start, bin_end : int, numéro du bin pour calculer la concentration que sur une gamme de taille précise (par exemple entre 150 et 300, indiquer numéro du bin correspondant à 150 et à 300).
    return : pd.Series, index : temps, colonne = concentration sur la gamme de taille choisie.
    '''
    try :
        if len(bin_size) == 1:
            bin_size = bin_size[0]
            concentration = (psd.iloc[:,bin_start:bin_end]*(bin_size)).sum(axis=1)
        elif type(bin_size) != pd.core.series.Series:
            bin_size = pd.Series(bin_size)
            concentration = (psd.iloc[:,bin_start:bin_end]*(bin_size[bin_start:bin_end].values)).sum(axis=1)
        else : 
            concentration = (psd.iloc[:,bin_start:bin_end]*(bin_size[bin_start:bin_end].values)).sum(axis=1)
        return concentration 
    except:
        print('bin_size must be a list or a pd.core.series.Series')
    
def LWC(psd, bin_size, diameter, bin_start = None, bin_end = None):
    '''
    psd : pd.DataFrame, index : temps, colonne : bin.
    bin_size : pd.Series() ou list, permet de passer de #/Volume/taille à #/Volume. Correspond à la différence (upper_bin_edge-lower_bin_edge).
    diameter = pd.Series(), midbin pour chaque bin. 
    bin_start, bin_end : int, numéro du bin pour calculer la concentration que sur une gamme de taille précise (par exemple entre 150 et 300, indiquer numéro du bin correspondant à 150 et à 300).
    return : pd.Dataframe, index : temps, colonne = LWC/bin sur la gamme de taille choisie.
    '''
    try :
        if len(bin_size) == 1:
            bin_size = bin_size[0]
            lwc = (1e-9)*(np.pi/6) * (psd.iloc[:,bin_start:bin_end].mul(bin_size)).mul(np.tile(np.power(diameter[bin_start:bin_end],3), (len(psd),1)))
        elif type(bin_size) != pd.core.series.Series:
            bin_size = pd.Series(bin_size)
            lwc = (1e-9)*(np.pi/6) * (psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end],3), (len(psd),1)))
        else : 
            lwc = (1e-9)*(np.pi/6) * (psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end],3), (len(psd),1)))
        return lwc       
    except : 
        print('bin_size must be a list or a pd.core.series.Series')

def Extinction(psd, bin_size, diameter, bin_start = None, bin_end = None):
    '''
    psd : pd.DataFrame, index : temps, colonne : bin.
    bin_size : pd.Series() ou list, permet de passer de #/Volume/taille à #/Volume. Correspond à la différence (upper_bin_edge-lower_bin_edge).
    diameter = pd.Series(), midbin pour chaque bin. 
    bin_start, bin_end : int, numéro du bin pour calculer la concentration que sur une gamme de taille précise (par exemple entre 150 et 300, indiquer numéro du bin correspondant à 150 et à 300).
    return : pd.Dataframe, index : temps, colonne = Extinction/bin sur la gamme de taille choisie.
    '''
    try : 
        if len(bin_size) == 1:
            bin_size = bin_size[0]
            extinction = (1e-6) * (np.pi/2) * (psd.iloc[:,bin_start:bin_end].mul(bin_size)).mul(np.tile(np.power(diameter[bin_start:bin_end],2), (len(psd),1)))
        elif type(bin_size) != pd.core.series.Series:
            bin_size = pd.Series(bin_size)
            extinction = (1e-6) * (np.pi/2) *(psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end],2), (len(psd),1)))
        else : 
            extinction = (1e-6) * (np.pi/2) * (psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end],2), (len(psd),1)))
        return extinction 
    except : 
        print('bin_size must be a list or a pd.core.series.Series')
        
def Effective_Diameter(psd, bin_size, diameter, bin_start = None, bin_end = None):
    '''ONLY FOR LIQUID'''
    try : 
        if len(bin_size) == 1:
            bin_size = bin_size[0]
            d3 = np.sum((psd.iloc[:,bin_start:bin_end].mul(bin_size)).mul(np.tile(np.power(diameter[bin_start:bin_end],3), (len(psd),1))), axis=1)
            d2 = np.sum((psd.iloc[:,bin_start:bin_end].mul(bin_size)).mul(np.tile(np.power(diameter[bin_start:bin_end],2), (len(psd),1))), axis=1)
            d_eff = d3/d2
        elif type(bin_size) != pd.core.series.Series:
            bin_size = pd.Series(bin_size)
            d_eff = (1e-9)*(np.pi/6) * (psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end],3), (len(psd),1)))
        else : 
            d_eff = (1e-9)*(np.pi/6) * (psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end],3), (len(psd),1)))
        return d_eff
    except : 
        print('bin_size must be a list or a pd.core.series.Series')

## Remarque sur le calcul de l'Ice Water Content. 
Par rapport aux fonctions définies plus haut, deux nouveaux paramètres sont ajoutés : 
- alpha = float, defaut = 0.0121
- beta = float, defaut = 1.9

Ces paramètres correspondent au calcul de $m = \alpha D^{\beta}$. \
Quelques paramètres de loi masse diamètre : 
* $\alpha$ = 0.0121 & $\beta$ = 1.9 avec $D=D_{max}$ : Hogan (=B&F95 modifié pour être utilisé avec Dmax à la place de Dmean) (D en m et m en kg). 
* $\alpha$ = 0.005484 & $\beta$ = 2.14800 avec $D=D_{?}$ : Heymsfield (T = -30°C) (D en cm, m en g)
* $\alpha$ = 0.008210 & $\beta$ = 2.44908 avec $D=D_{?}$ : Yang Plaque (D en cm, m en g)
* $\alpha$ = 0.086534 & $\beta$ = 2.77712 avec $D=D_{?}$ : Yang Colonne(D en cm, m en g)
* $\alpha$ = 0.004834 & $\beta$ = 2.50649 avec $D=D_{?}$ : Yang Bullet-6 (D en cm, m en g)
* $\alpha$ = 0.497345 & $\beta$ = 3.29561 avec $D=D_{?}$ : Yang Mixte (D en cm, m en g)
* $\alpha$ = 0.0081e^{0.013T} & $\beta$ = 2.31 + 0.0054T avec $D=D_{?}$ : Heymsfield (T dépendant) (D en cm, m en g)

In [ ]:
def IWC(psd, bin_size, diameter, alpha = 0.0121, beta = 1.9, bin_start = None, bin_end = None):
    '''
    psd : pd.DataFrame, index : temps, colonne : bin.
    bin_size : pd.Series() ou list, permet de passer de #/Volume/taille à #/Volume. Correspond à la différence (upper_bin_edge-lower_bin_edge).
    diameter = pd.Series(), midbin pour chaque bin. 
    bin_start, bin_end : int, numéro du bin pour calculer la concentration que sur une gamme de taille précise (par exemple entre 150 et 300, indiquer numéro du bin correspondant à 150 et à 300).
    return : pd.Dataframe, index : temps, colonne = IWC/bin sur la gamme de taille choisie.
    '''
    try : 
        if len(bin_size) == 1:
            bin_size = bin_size[0]
            iwc = (alpha*psd.iloc[:,bin_start:bin_end].mul(bin_size)).mul(np.tile(np.power(diameter[bin_start:bin_end]*1e-6,beta), (len(psd),1)))
        elif type(bin_size) != pd.core.series.Series:
            bin_size = pd.Series(bin_size)
            iwc = (alpha*psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end]*1e-6,beta), (len(psd),1)))
        else : 
            iwc = (alpha*psd.iloc[:,bin_start:bin_end].mul(bin_size[bin_start:bin_end].values)).mul(np.tile(np.power(diameter[bin_start:bin_end]*1e-6,beta), (len(psd),1)))
        return iwc * 1e6
    except : 
        print('bin_size must be a list or a pd.core.series.Series')

## MMD/MVD 
Cette fonction nécessite d'avoir les milieux (ou bord gauche ou bord droit) de bin en nom de colonne. \
La fonction prend en entrée 3 paramètres obligatoire :
- ***psd*** : pandas.core.frame.DataFrame, *PSD en **#/L** à partir de laquelle calculer le paramètre dérivé.*
- ***option*** : str, defaut = 'MMD'. *'MMD' si solide ou 'MVD' si liquide.*
- ***iwc/lwc*** : pandas.core.frame.DataFrame. *iwc en $g.m^{-3}/bin$ si option == 'MMD', lwc en $g.m^{-3}/bin$ si option == 'MVD'*

La fonction prends en entrée 1 paramètre optionnel : 
- ***plot***, boolean, defaut = False. *Pour ploter le MMD, peux être très long en temps.*

In [ ]:
def MMD(psd, option = 'MMD', plot=False, iwc=None, lwc = None):
    if option == 'MMD':
        pmd = iwc.T
        #MMD - 
        cumsum = pmd.cumsum(axis=0).T
        mask = cumsum.sum(axis=1) == 0
        mmd = (abs(pmd.cumsum(axis=0).sub(pmd.sum(axis=0)/2))).T
        MMD = mmd.T.idxmin() 
        MMD[(mask)] = np.nan
        
        if plot == True : 
            import matplotlib.pyplot as plt
            fig, ax =plt.subplots(figsize=[10,10], dpi=300)
            mmd.iloc[5000:6000,:].T.plot(ax=ax, legend=False)
            ax.set_xlim(0,1280)
            plt.show()
            
        return MMD
    elif option == 'MVD' : 
        #Computation of LWC
        pmd = lwc.T
        #MVD -
        cumsum = pmd.cumsum(axis=0).T
        mask = cumsum.sum(axis=1) == 0
        mvd = (abs(pmd.cumsum(axis=0).sub(pmd.sum(axis=0)/2))).T
        MVD = mvd.T.idxmin() 
        MVD[(mask)] = np.nan
        
        if plot == True : 
            import matplotlib.pyplot as plt
            fig, ax =plt.subplots(figsize=[10,10], dpi=300)
            mvd.iloc[5000:6000,:].T.plot(ax=ax, legend=False)
            ax.set_xlim(0,500)
            plt.show()
            
        return MVD

DS_circularity_dmax.columns = np.arange(15,1295,10)    
test = MMD(DS_circularity_dmax, option='MMD', plot=False, iwc = iwc)